# required packages

In [2]:
%%time
#downloading video using pytube
from pytube import YouTube
#directory
import os
#converting video to audio using moviepy
import moviepy.editor as mp
import pandas as pd
import glob
import sys

CPU times: user 986 ms, sys: 223 ms, total: 1.21 s
Wall time: 950 ms


# in specified directory all conversion will be done

In [3]:
VidDir="/home/ckpl/Downloads/Voice2txt"

# passing all VidLink in dataframe

In [10]:
data={'VidLink':['https://youtu.be/ZxKfQVxlE9w']}
#data = {'VidLink':  ['https://youtu.be/NNOZ-cjsAT0','https://youtu.be/QsynaMuQhZg','https://youtu.be/Q8OJGd54B9Q','https://youtu.be/HhcRV2Fe3Fg', 'https://youtu.be/rITo2zDNM8M','https://youtu.be/rITo2zDNM8M','https://youtu.be/PH9z9LdjQFk','https://youtu.be/fQypGNWCYbs','https://youtu.be/0nBzGIxJEtM','https://youtu.be/C38iGUSvBCA','https://youtu.be/NQzAEBNwC3k']}
df = pd.DataFrame (data, columns = ['VidLink'])
print (df)

                        VidLink
0  https://youtu.be/ZxKfQVxlE9w


# youtube key will be extracted from the youtube link

In [ ]:
df['key']=''
df['key']= df['VidLink'].apply(lambda x: x[x.find("https://youtu.be/")+17:])


for i in df['VidLink']: 
    try:
        yt_obj = YouTube(i)
        #df['Title']=yt_obj.title
        print(f'Title:{yt_obj.title}')
        #print(f'Length:{yt_obj.length}')
        #print(f'Rating:{yt_obj.rating}')
        #print(f'ViewsCount:{yt_obj.views}')
        #print(f'Author:{yt_obj.author}')
    except Exception as e:
        print(e)

# function for download vid,aud convert,concate str in a list

In [8]:
%%time
def concatenate_list_data(list):
    result= ''
    for element in list:
        result += str(element)
    return result

def downloadYouTube(videourl, path):
    #DownloadYoutube
    yt = YouTube(videourl)
    yt = yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first()
    if not os.path.exists(path):
        os.makedirs(path)
    yt.download(path)

def Video2Audio():
    vidset = [file for file in glob.glob(VidDir + "**/*.mp4", recursive=True)]
    for file in vidset:
        video=mp.VideoFileClip(file)
        newfile=file[:-4]
        video.audio.write_audiofile(newfile+".wav")

CPU times: user 7 µs, sys: 1e+03 ns, total: 8 µs
Wall time: 10.5 µs


# passing entire VidLink for conversion

In [11]:
%%time
#link to video conversion
for i in  df['VidLink']:
    downloadYouTube(i,VidDir)
#vidtoaud conversion
Video2Audio()

chunk:   4%|▍         | 255/6528 [00:00<00:02, 2451.13it/s, now=None]

MoviePy - Writing audio in /home/ckpl/Downloads/Voice2txt/2 Simple Face Mask For Glowing And Ageless Skin.wav


chunk:   0%|          | 0/11767 [00:00<?, ?it/s, now=None]            

MoviePy - Done.
MoviePy - Writing audio in /home/ckpl/Downloads/Voice2txt/Top 5 HAIR CARE Tips for Indian Men  Mens Hair Care Routine  BeerBiceps Grooming.wav


chunk:   0%|          | 0/4658 [00:00<?, ?it/s, now=None]               

...

chunk:   0%|          | 0/717 [00:00<?, ?it/s, now=None]              

MoviePy - Done.
MoviePy - Writing audio in /home/ckpl/Downloads/Voice2txt/Clean & Clears Foaming Face Wash (English).wav


chunk:   0%|          | 0/15342 [00:00<?, ?it/s, now=None]          

MoviePy - Done.
MoviePy - Writing audio in /home/ckpl/Downloads/Voice2txt/The secret to my GLOWING skin  Product Review  Chetali Chadha.wav


MoviePy - Done.
CPU times: user 14.3 s, sys: 979 ms, total: 15.3 s
Wall time: 24.8 s


# std custom endpoint,speech key,region r passed in variable

In [ ]:
%%time
import time
import wave
try:
    import azure.cognitiveservices.speech as speechsdk
except ImportError:
    print("""
    Importing the Speech SDK for Python failed.
    Refer to
    https://docs.microsoft.com/azure/cognitive-services/speech-service/quickstart-python for
    installation instructions.
    """)
    import sys
    sys.exit(1)

speech_key, service_region = "0f95674905f94c32abfe87a2679aa37d", "eastus"

EndpointId="a1679626-4779-43ba-8c6a-876722f5601f"

# function for continous speech recognisation 

In [ ]:
%%time
def speech_recognize_continuous_from_file(file):
    """performs continuous speech recognition with input from an audio file"""
    # <SpeechContinuousRecognitionWithFile>
    speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=service_region)
    speech_config.endpoint_id = EndpointId
    audio_config = speechsdk.audio.AudioConfig(filename=file)

    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)

    done = False

    def stop_cb(evt):
        """callback that stops continuous recognition upon receiving an event `evt`"""
        #print('CLOSING on {}'.format(evt))
        speech_recognizer.stop_continuous_recognition()
        nonlocal done
        done = True

    all_results = []
    def handle_final_result(evt):
        all_results.append(evt.result.text)
        
    speech_recognizer.recognized.connect(handle_final_result)
    # Connect callbacks to the events fired by the speech recognizer
    speech_recognizer.recognizing.connect(lambda evt: 'RECOGNIZING: {}'.format(evt))
    speech_recognizer.recognized.connect(lambda evt: 'RECOGNIZED: {}'.format(evt))
    speech_recognizer.session_started.connect(lambda evt: 'SESSION STARTED: {}'.format(evt))
    speech_recognizer.session_stopped.connect(lambda evt: 'SESSION STOPPED {}'.format(evt))
    speech_recognizer.canceled.connect(lambda evt: 'CANCELED {}'.format(evt))
    # stop continuous recognition on either session stopped or canceled events
    speech_recognizer.session_stopped.connect(stop_cb)
    speech_recognizer.canceled.connect(stop_cb)

    # Start continuous speech recognition
    speech_recognizer.start_continuous_recognition()
    while not done:
        time.sleep(.5)    
    return all_results

# Loop to call function to convert audio files to text

In [ ]:
%%time

output_list = []

fileset = [file for file in glob.glob(VidDir + "**/*.wav", recursive=True)]
temp = [speech_recognize_continuous_from_file(file) for file in fileset]

for i in range(0,len(temp)):
    output_list.append(concatenate_list_data(temp[i]))
    
output_df = pd.DataFrame(output_list,columns=['text'])

In [ ]:
[x for x in output_df['text']]

# passing the text to ingredient model

In [ ]:
# Load pre-existing spacy model
import spacy
import re
prdnlp=spacy.load('/home/ckpl/Downloads/utube_en_ing/prdnlpOct20')

# Getting the pipeline component
#ner=nlp.get_pipe("ner")

In [ ]:
# Save our trained Model
modelfile = "prdnlp"
prdnlp.to_disk(modelfile)

In [ ]:
output_df=pd.read_csv("output_df.csv")

In [ ]:
output_df

In [ ]:
#required attribute - dashboard
del output_df['Unnamed: 0']

In [ ]:
output_df = output_df[['text']].copy()
output_df['text']=output_df['text'].str.replace("[^a-zA-Z#]", " ")
output_df['text']=[x for x in output_df['text'].str.lower().str.strip()]
output_df['text']=output_df['text'].apply(lambda x : (re.sub('\s+',' ',x)))

In [ ]:
output_df['ing']=""
for index,row in output_df.iterrows():
    doc=prdnlp(row['text'])
    row['ing']={ent.text for ent in doc.ents}
    output_df['ing'].append(print(row['ing']))

In [ ]:
output_df

# extracting filename for Title column

In [ ]:
import os

def file_base_name(file_name):
    if '.' in file_name:
        separator_index = file_name.index('.')
        base_name = file_name[:separator_index]
        return base_name
    else:
        return file_name

def path_base_name(VidDir):
    file_name = os.path.basename(VidDir)
    return file_base_name(file_name)

In [ ]:
parsed_text={'title':[]}
fileset = [file for file in glob.glob(VidDir + "**/*.wav", recursive=True)]
for file in fileset:
    parsed_text['title'].append(path_base_name(file))
output_df['Title']=parsed_text['title']

In [ ]:
output_df

In [ ]:
for i in df['VidLink']: 
    try:
        yt_obj = YouTube(i)
        print(f'Title:{yt_obj.title}')
        print(f'Length:{yt_obj.length}')
        print(f'Rating:{yt_obj.rating}')
        print(f'ViewsCount:{yt_obj.views}')
        print(f'Author:{yt_obj.author}')
    except Exception as e:
        print(e)

In [ ]:
from __future__ import print_function
from typing import List

import logging
import sys
import requests
import time
import swagger_client as cris_client

In [ ]:
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG, format="%(message)s")

SUBSCRIPTION_KEY = "891818643be44c078a57ee2ced246297"

HOST_NAME = "westeurope.cris.ai"
PORT = 443

NAME = "Simple transcription"
DESCRIPTION = "Simple transcription description"

LOCALE = "en-IN"
RECORDINGS_BLOB_URI = "/home/ckpl/Documents/multi.wav"
# ADAPTED_ACOUSTIC_ID = None  # guid of a custom acoustic model
# ADAPTED_LANGUAGE_ID = None  # guid of a custom language model

In [ ]:
def transcribe():
    logging.info("Starting transcription client...")

    # configure API key authorization: subscription_key
    configuration = cris_client.Configuration()
    configuration.api_key['Ocp-Apim-Subscription-Key'] = SUBSCRIPTION_KEY

    # create the client object and authenticate
    client = cris_client.ApiClient(configuration)

    # create an instance of the transcription api class
    transcription_api = cris_client.CustomSpeechTranscriptionsApi(api_client=client)

    # get all transcriptions for the subscription
    transcriptions: List[cris_client.Transcription] = transcription_api.get_transcriptions()

    logging.info("Deleting all existing completed transcriptions.")

    # delete all pre-existing completed transcriptions
    # if transcriptions are still running or not started, they will not be deleted
    for transcription in transcriptions:
        transcription_api.delete_transcription(transcription.id)

    logging.info("Creating transcriptions.")

    # transcription definition using custom models
#     transcription_definition = cris_client.TranscriptionDefinition(
#         name=NAME, description=DESCRIPTION, locale=LOCALE, recordings_url=RECORDINGS_BLOB_URI,
#         models=[cris_client.ModelIdentity(ADAPTED_ACOUSTIC_ID), cris_client.ModelIdentity(ADAPTED_LANGUAGE_ID)]
#     )

    # comment out the previous statement and uncomment the following to use base models for transcription
    transcription_definition = cris_client.TranscriptionDefinition(
         name=NAME, description=DESCRIPTION, locale=LOCALE, recordings_url=RECORDINGS_BLOB_URI
     )

    data, status, headers = transcription_api.create_transcription_with_http_info(transcription_definition)

    # extract transcription location from the headers
    transcription_location: str = headers["location"]

    # get the transcription Id from the location URI
    created_transcriptions = list()
    created_transcriptions.append(transcription_location.split('/')[-1])

    logging.info("Checking status.")

    completed, running, not_started = 0, 0, 0

    while completed < 1:
        # get all transcriptions for the user
        transcriptions: List[cris_client.Transcription] = transcription_api.get_transcriptions()

        # for each transcription in the list we check the status
        for transcription in transcriptions:
            if transcription.status == "Failed" or transcription.status == "Succeeded":
                # we check to see if it was one of the transcriptions we created from this client
                if transcription.id not in created_transcriptions:
                    continue

                completed += 1

                if transcription.status == "Succeeded":
                    results_uri = transcription.results_urls["channel_0"]
                    results = requests.get(results_uri)
                    logging.info("Transcription succeeded. Results: ")
                    logging.info(results.content.decode("utf-8"))
            elif transcription.status == "Running":
                running += 1
            elif transcription.status == "NotStarted":
                not_started += 1

        logging.info(f"Transcriptions status: {completed} completed, {running} running, {not_started} not started yet")
        # wait for 5 seconds
        time.sleep(5)

    input("Press any key...")


def main():
    transcribe()


if __name__ == "__main__":
    main()

# Issue to fix
#when compared with google/azure python continous/azure front page js speech to txt app
#azure front page js speech to txt app gives better good output
#so wanna improve the accuracy as lyk in azure app
#diarization is a property used to identify different speakers in a txt in batch (how to add wanna c)